In [1]:
import geopandas as gpd
import pandas as pd
import pickle
import momepy
import dask
from dask.distributed import Client, LocalCluster
from tqdm.auto import tqdm
from scripts.percentile import Percentiles


/tmp/ipykernel_4970/1586022436.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
local_crs = 3414

place = "singapore"

In [3]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster, timeout="300s")

client



/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46173 instead
  warnings.warn(
2023-04-27 16:38:28,946 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:38:28,948 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:38:28,950 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:38:28,953 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:38:28,956 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:38:28,959 - distributed.nanny.memory - WARNING - Ignoring

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46173/status,
Dashboard: http://127.0.0.1:46173/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36483,Workers: 8
Dashboard: http://127.0.0.1:46173/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:45793,Total threads: 2
Dashboard: http://127.0.0.1:34727/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:42561,


In [4]:
tessellation = gpd.read_parquet(f"./out/{place}/tessellation_stats.pq")
buildings = gpd.read_parquet(f"./out/{place}/buildings.pq")

with open(f"./out/{place}/weights.pickle", 'rb') as f:
    queen_1 = pickle.load(f)
    queen_3 = pickle.load(f)
    dist200 = pickle.load(f)
    f.close()
    

In [5]:
percentiles = []
for column in tqdm(tessellation.columns.drop(["geometry", "uID"]), total = len(tessellation.columns.drop(["geometry", "uID"]))):
    perc = Percentiles(tessellation, column, queen_3, "uID", verbose=True, client=client).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)
    
percentiles.append(tessellation[["uID", "geometry"]])

  0%|          | 0/37 [00:00<?, ?it/s]

<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 11.16 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
<string>:1: UserWarning: Shapely 2.0 is installed, bu

KeyboardInterrupt: 

2023-04-27 16:37:39,095 - distributed.worker - WARNING - Compute Failed
Key:       process-130c1725-d707-4592-9ca4-74cb3a69ea62
Function:  execute_task
args:      ((<function Percentiles.__init__.<locals>.process at 0x7fa9f3f39ab0>, 73797.0, <libpysal.weights.weights.W object at 0x7faa0848bd00>, uID
75707.0    0.957765
74906.0    0.840004
74909.0    0.982273
74915.0    0.941257
74921.0    0.921910
             ...   
74471.0    0.936044
75817.0    0.653799
75833.0    0.244914
75840.0    0.991706
75848.0    1.000000
Name: convexity, Length: 76498, dtype: float64, [25, 50, 75], (<class 'dict'>, [['method', 'midpoint']])))
kwargs:    {}
Exception: "KeyError('[30225.0] not in index')"

2023-04-27 16:37:39,108 - distributed.worker - WARNING - Compute Failed
Key:       process-28ff077e-c8f8-400a-9e60-b651427599b0
Function:  execute_task
args:      ((<function Percentiles.__init__.<locals>.process at 0x7f628a971900>, 75597.0, <libpysal.weights.weights.W object at 0x7f629389b790>, uID
75707.0 

In [ ]:
percentiles_joined = pd.concat(percentiles, axis=1)
percentiles_joined.head()

In [ ]:
percentiles_joined = gpd.GeoDataFrame(percentiles_joined, geometry="geometry", crs=local_crs)

In [ ]:
print(percentiles_joined.columns.tolist())

In [ ]:
percentiles_joined.to_parquet(f"./out/{place}/tessellation_percentiles.pq")